In [2]:
# ========================================================================
# NEURAL NETWORKS + ADVANCED STACKING PIPELINE
# Goal: Push from 0.6385 → 0.65+ without overfitting
# Strategy: Add diverse models + better stacking architecture
# ========================================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from scipy.stats import rankdata

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier

import joblib

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


print("=" * 80)
print("NEURAL NETWORKS + ADVANCED STACKING PIPELINE")
print("Current Best: 0.6385 → Target: 0.65+")
print("=" * 80)

NEURAL NETWORKS + ADVANCED STACKING PIPELINE
Current Best: 0.6385 → Target: 0.65+


In [3]:
# ========================================================================
# STEP 1: LOAD PREPROCESSED DATA
# ========================================================================
print("\n" + "=" * 80)
print("STEP 1: LOADING DATA")
print("=" * 80)

train = pd.read_csv('train1.csv')
test = pd.read_csv('test.csv')

TARGET = 'target'
ID_COL = 'id'

# Conservative feature engineering
def create_robust_features(df, is_train=True):
    """Minimal, stable features only"""
    df = df.copy()
    
    # Core interactions
    if 'ps_car_13' in df.columns and 'ps_reg_03' in df.columns:
        df['car13_reg03'] = df['ps_car_13'] * df['ps_reg_03']
    
    if 'ps_ind_15' in df.columns and 'ps_reg_01' in df.columns:
        df['ind15_reg01'] = df['ps_ind_15'] * df['ps_reg_01']
    
    # Simple aggregations
    car_cols = [c for c in df.columns if c.startswith('ps_car_') and c.endswith('_cat')]
    if car_cols:
        df['car_cat_sum'] = df[car_cols].sum(axis=1)
    
    ind_cols = [c for c in df.columns if c.startswith('ps_ind_') and c.endswith('_bin')]
    if ind_cols:
        df['ind_bin_sum'] = df[ind_cols].sum(axis=1)
    
    # Key missing indicators
    for col in ['ps_car_03_cat', 'ps_car_05_cat', 'ps_reg_03']:
        if col in df.columns:
            df[f'{col}_missing'] = df[col].isnull().astype(int)
    
    return df

train_fe = create_robust_features(train, is_train=True)
test_fe = create_robust_features(test, is_train=False)

X_full = train_fe.drop([TARGET, ID_COL], axis=1)
y_full = train_fe[TARGET]
test_full = test_fe.drop([ID_COL], axis=1, errors='ignore')

# Align columns
common_features = list(set(X_full.columns) & set(test_full.columns))
X_full = X_full[common_features]
test_full = test_full[common_features]

# Fill missing
X_full = X_full.fillna(-999)
test_full = test_full.fillna(-999)

print(f"Features: {X_full.shape[1]}")
print(f"Samples: {X_full.shape[0]}")


STEP 1: LOADING DATA
Features: 72
Samples: 296209


In [4]:
# ========================================================================
# STEP 2: PREPARE SCALED DATA FOR NEURAL NETWORKS
# ========================================================================
print("\n" + "=" * 80)
print("STEP 2: PREPARING SCALED DATA")
print("=" * 80)

# Neural networks need scaling
scaler = StandardScaler()
X_full_scaled = scaler.fit_transform(X_full)
test_full_scaled = scaler.transform(test_full)

print("✅ Data scaled for neural networks")


STEP 2: PREPARING SCALED DATA


✅ Data scaled for neural networks


In [5]:
# ========================================================================
# STEP 3: DEFINE DIVERSE BASE MODELS
# ========================================================================
print("\n" + "=" * 80)
print("STEP 3: TRAINING DIVERSE BASE MODELS")
print("=" * 80)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# Storage for OOF predictions
oof_predictions = {}
test_predictions = {}

# -------- Model 1: CatBoost --------
print("\n📊 1/7: CatBoost...")
oof_cat = np.zeros(len(X_full))
test_cat = np.zeros(len(test_full))

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_full, y_full)):
    print(f"  Fold {fold+1}/5", end=' ')
    
    X_tr, X_vl = X_full.iloc[train_idx], X_full.iloc[val_idx]
    y_tr, y_vl = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    model = CatBoostClassifier(
        iterations=500,
        depth=5,
        learning_rate=0.03,
        l2_leaf_reg=9,
        random_strength=2,
        bagging_temperature=0.8,
        auto_class_weights='Balanced',
        random_state=RANDOM_SEED + fold,
        verbose=0
    )
    
    model.fit(X_tr, y_tr, eval_set=(X_vl, y_vl), early_stopping_rounds=50, verbose=False)
    
    oof_cat[val_idx] = model.predict_proba(X_vl)[:, 1]
    test_cat += model.predict_proba(test_full)[:, 1] / 5
    print(f"AUC: {roc_auc_score(y_vl, oof_cat[val_idx]):.4f}")

oof_predictions['catboost'] = oof_cat
test_predictions['catboost'] = test_cat
print(f"✅ CatBoost OOF AUROC: {roc_auc_score(y_full, oof_cat):.4f}")

# -------- Model 2: LightGBM --------
print("\n📊 2/7: LightGBM...")
oof_lgb = np.zeros(len(X_full))
test_lgb = np.zeros(len(test_full))

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_full, y_full)):
    print(f"  Fold {fold+1}/5", end=' ')
    
    X_tr, X_vl = X_full.iloc[train_idx], X_full.iloc[val_idx]
    y_tr, y_vl = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    model = lgb.LGBMClassifier(
        n_estimators=500,
        num_leaves=40,
        learning_rate=0.03,
        min_child_samples=40,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.5,
        reg_lambda=0.5,
        class_weight='balanced',
        random_state=RANDOM_SEED + fold,
        verbose=-1
    )
    
    model.fit(X_tr, y_tr, eval_set=[(X_vl, y_vl)], 
              callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
    
    oof_lgb[val_idx] = model.predict_proba(X_vl)[:, 1]
    test_lgb += model.predict_proba(test_full)[:, 1] / 5
    print(f"AUC: {roc_auc_score(y_vl, oof_lgb[val_idx]):.4f}")

oof_predictions['lightgbm'] = oof_lgb
test_predictions['lightgbm'] = test_lgb
print(f"✅ LightGBM OOF AUROC: {roc_auc_score(y_full, oof_lgb):.4f}")

# -------- Model 3: XGBoost --------
print("\n📊 3/7: XGBoost...")
oof_xgb = np.zeros(len(X_full))
test_xgb = np.zeros(len(test_full))

scale_pos_weight = (y_full == 0).sum() / (y_full == 1).sum()

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_full, y_full)):
    print(f"  Fold {fold+1}/5", end=' ')
    
    X_tr, X_vl = X_full.iloc[train_idx], X_full.iloc[val_idx]
    y_tr, y_vl = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    model = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.03,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        gamma=1.0,
        reg_alpha=0.5,
        reg_lambda=0.5,
        scale_pos_weight=scale_pos_weight,
        random_state=RANDOM_SEED + fold
    )
    
    from xgboost.callback import EarlyStopping

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_vl, y_vl)]
    )

    
    oof_xgb[val_idx] = model.predict_proba(X_vl)[:, 1]
    test_xgb += model.predict_proba(test_full)[:, 1] / 5
    print(f"AUC: {roc_auc_score(y_vl, oof_xgb[val_idx]):.4f}")

oof_predictions['xgboost'] = oof_xgb
test_predictions['xgboost'] = test_xgb
print(f"✅ XGBoost OOF AUROC: {roc_auc_score(y_full, oof_xgb):.4f}")

# -------- Model 4: ExtraTrees (for diversity) --------
print("\n📊 4/7: ExtraTrees...")
oof_et = np.zeros(len(X_full))
test_et = np.zeros(len(test_full))

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_full, y_full)):
    print(f"  Fold {fold+1}/5", end=' ')
    
    X_tr, X_vl = X_full.iloc[train_idx], X_full.iloc[val_idx]
    y_tr, y_vl = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    model = ExtraTreesClassifier(
        n_estimators=300,
        max_depth=8,
        min_samples_leaf=5,
        class_weight='balanced',
        random_state=RANDOM_SEED + fold,
        n_jobs=1
    )
    
    model.fit(X_tr, y_tr)
    
    oof_et[val_idx] = model.predict_proba(X_vl)[:, 1]
    test_et += model.predict_proba(test_full)[:, 1] / 5
    print(f"AUC: {roc_auc_score(y_vl, oof_et[val_idx]):.4f}")

oof_predictions['extratrees'] = oof_et
test_predictions['extratrees'] = test_et
print(f"✅ ExtraTrees OOF AUROC: {roc_auc_score(y_full, oof_et):.4f}")

# -------- Model 5: MLPClassifier (Neural Network) --------
print("\n📊 5/7: MLPClassifier (Neural Network)...")
oof_mlp = np.zeros(len(X_full))
test_mlp = np.zeros(len(test_full))

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_full, y_full)):
    print(f"  Fold {fold+1}/5", end=' ')
    
    X_tr_scaled = X_full_scaled[train_idx]
    X_vl_scaled = X_full_scaled[val_idx]
    y_tr, y_vl = y_full.iloc[train_idx], y_full.iloc[val_idx]
    
    model = MLPClassifier(
        hidden_layer_sizes=(128, 64, 32),
        activation='relu',
        solver='adam',
        alpha=0.01,  # L2 regularization
        batch_size=256,
        learning_rate='adaptive',
        learning_rate_init=0.001,
        max_iter=100,
        early_stopping=True,
        validation_fraction=0.1,
        random_state=RANDOM_SEED + fold,
        verbose=False
    )
    
    model.fit(X_tr_scaled, y_tr)
    
    oof_mlp[val_idx] = model.predict_proba(X_vl_scaled)[:, 1]
    test_mlp += model.predict_proba(test_full_scaled)[:, 1] / 5
    print(f"AUC: {roc_auc_score(y_vl, oof_mlp[val_idx]):.4f}")

oof_predictions['mlp'] = oof_mlp
test_predictions['mlp'] = test_mlp
print(f"✅ MLP OOF AUROC: {roc_auc_score(y_full, oof_mlp):.4f}")

# -------- Model 6: TabNet (if available) --------


# -------- Model 7: Custom PyTorch NN (if available) --------



STEP 3: TRAINING DIVERSE BASE MODELS

📊 1/7: CatBoost...
  Fold 1/5 AUC: 0.6390
  Fold 2/5 AUC: 0.6391
  Fold 3/5 AUC: 0.6302
  Fold 4/5 AUC: 0.6407
  Fold 5/5 AUC: 0.6311
✅ CatBoost OOF AUROC: 0.6360

📊 2/7: LightGBM...
  Fold 1/5 

  File "C:\Users\SHEIKHANI LAPTOP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 247, in _count_physical_cores
    cpu_count_physical = _count_physical_cores_win32()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SHEIKHANI LAPTOP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 299, in _count_physical_cores_win32
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.597772
AUC: 0.6330
  Fold 2/5 Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.598659
AUC: 0.6306
  Fold 3/5 Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.598854
AUC: 0.6234
  Fold 4/5 Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.597835
AUC: 0.6316
  Fold 5/5 Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's binary_logloss: 0.59906
AUC: 0.6246
✅ LightGBM OOF AUROC: 0.6286

📊 3/7: XGBoost...
  Fold 1/5 [0]	validation_0-logloss:0.69201
[1]	validation_0-logloss:0.69072
[2]	validation_0-loglo

In [6]:
# ========================================================================
# STEP 4: BASE MODEL SUMMARY
# ========================================================================
print("\n" + "=" * 80)
print("BASE MODEL SUMMARY")
print("=" * 80)

base_results = []
for name, preds in oof_predictions.items():
    auc = roc_auc_score(y_full, preds)
    base_results.append({'Model': name, 'OOF AUROC': auc})
    print(f"{name:15s}: {auc:.4f}")

base_df = pd.DataFrame(base_results).sort_values('OOF AUROC', ascending=False)


BASE MODEL SUMMARY
catboost       : 0.6360
lightgbm       : 0.6286
xgboost        : 0.6324
extratrees     : 0.6146
mlp            : 0.5973


In [7]:
# ========================================================================
# STEP 5: ADVANCED STACKING STRATEGIES
# ========================================================================
print("\n" + "=" * 80)
print("STEP 5: ADVANCED STACKING")
print("=" * 80)

# Prepare stacking features
oof_stack = np.column_stack([oof_predictions[name] for name in oof_predictions.keys()])
test_stack = np.column_stack([test_predictions[name] for name in test_predictions.keys()])

print(f"Stacking features shape: {oof_stack.shape}")

# -------- Strategy 1: Ridge-regularized Logistic Regression --------
print("\n1️⃣  Ridge Logistic Regression...")
from sklearn.linear_model import LogisticRegressionCV

lr_cv = LogisticRegressionCV(
    Cs=np.logspace(-4, 4, 20),
    cv=5,
    scoring='roc_auc',
    class_weight='balanced',
    random_state=RANDOM_SEED,
    max_iter=1000,
    n_jobs=-1
)
lr_cv.fit(oof_stack, y_full)

oof_lr = lr_cv.predict_proba(oof_stack)[:, 1]
test_lr = lr_cv.predict_proba(test_stack)[:, 1]
lr_auc = roc_auc_score(y_full, oof_lr)

print(f"   Best C: {lr_cv.C_[0]:.4f}")
print(f"   OOF AUROC: {lr_auc:.4f}")

# -------- Strategy 2: LightGBM Meta-model --------
print("\n2️⃣  LightGBM Meta-model...")
lgb_meta = lgb.LGBMClassifier(
    n_estimators=100,
    num_leaves=15,  # Very shallow
    learning_rate=0.05,
    min_child_samples=50,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=1.0,
    reg_lambda=1.0,
    class_weight='balanced',
    random_state=RANDOM_SEED,
    verbose=-1
)

# Use nested CV for meta-model
oof_lgb_meta = np.zeros(len(X_full))

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_full, y_full)):
    lgb_meta.fit(
        oof_stack[train_idx], y_full.iloc[train_idx],
        eval_set=[(oof_stack[val_idx], y_full.iloc[val_idx])],
        callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
    )
    oof_lgb_meta[val_idx] = lgb_meta.predict_proba(oof_stack[val_idx])[:, 1]

# Retrain on full data
lgb_meta.fit(oof_stack, y_full)
test_lgb_meta = lgb_meta.predict_proba(test_stack)[:, 1]
lgb_meta_auc = roc_auc_score(y_full, oof_lgb_meta)

print(f"   OOF AUROC: {lgb_meta_auc:.4f}")

# -------- Strategy 3: Neural Network Meta-model --------
print("\n3️⃣  Neural Network Meta-model...")

meta_nn = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    solver='adam',
    alpha=0.1,  # Strong regularization
    batch_size=256,
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    validation_fraction=0.15,
    random_state=RANDOM_SEED,
    verbose=False
)

meta_nn.fit(oof_stack, y_full)

oof_nn_meta = meta_nn.predict_proba(oof_stack)[:, 1]
test_nn_meta = meta_nn.predict_proba(test_stack)[:, 1]
nn_meta_auc = roc_auc_score(y_full, oof_nn_meta)

print(f"   OOF AUROC: {nn_meta_auc:.4f}")

# -------- Strategy 4: Rank Average --------
print("\n4️⃣  Rank Average...")

oof_ranks = np.column_stack([rankdata(oof_predictions[name]) for name in oof_predictions.keys()])
test_ranks = np.column_stack([rankdata(test_predictions[name]) for name in test_predictions.keys()])

oof_rank_avg = oof_ranks.mean(axis=1)
test_rank_avg = test_ranks.mean(axis=1)
rank_avg_auc = roc_auc_score(y_full, oof_rank_avg)

print(f"   OOF AUROC: {rank_avg_auc:.4f}")

# -------- Strategy 5: Weighted Average by Performance --------
print("\n5️⃣  Performance-weighted Average...")

weights = np.array([roc_auc_score(y_full, oof_predictions[name]) for name in oof_predictions.keys()])
weights = weights / weights.sum()

oof_weighted = oof_stack @ weights
test_weighted = test_stack @ weights
weighted_auc = roc_auc_score(y_full, oof_weighted)

print(f"   Weights: {dict(zip(oof_predictions.keys(), np.round(weights, 3)))}")
print(f"   OOF AUROC: {weighted_auc:.4f}")

# -------- Strategy 6: Optimized Weighted Average --------
print("\n6️⃣  Optimized Weights (Nelder-Mead)...")

from scipy.optimize import minimize

def neg_auc(weights, oof_stack, y_true):
    weights = np.abs(weights)
    weights = weights / weights.sum()
    pred = oof_stack @ weights
    return -roc_auc_score(y_true, pred)

initial_weights = np.ones(len(oof_predictions)) / len(oof_predictions)

result = minimize(
    neg_auc,
    initial_weights,
    args=(oof_stack, y_full),
    method='Nelder-Mead',
    options={'maxiter': 1000}
)

opt_weights = np.abs(result.x)
opt_weights = opt_weights / opt_weights.sum()

oof_optimized = oof_stack @ opt_weights
test_optimized = test_stack @ opt_weights
optimized_auc = roc_auc_score(y_full, oof_optimized)

print(f"   Optimized Weights: {dict(zip(oof_predictions.keys(), np.round(opt_weights, 3)))}")
print(f"   OOF AUROC: {optimized_auc:.4f}")


STEP 5: ADVANCED STACKING
Stacking features shape: (296209, 5)

1️⃣  Ridge Logistic Regression...
   Best C: 1.6238
   OOF AUROC: 0.6370

2️⃣  LightGBM Meta-model...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.655986
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.661213
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.658665
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.655069
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.65956
   OOF AUROC: 0.6351

3️⃣  Neural Network Meta-model...
   OOF AUROC: 0.6357

4️⃣  R

In [8]:
# ========================================================================
# STEP 6: FINAL ENSEMBLE COMPARISON
# ========================================================================
print("\n" + "=" * 80)
print("FINAL ENSEMBLE COMPARISON")
print("=" * 80)

ensemble_results = {
    'Ridge Logistic Regression': (lr_auc, test_lr),
    'LightGBM Meta-model': (lgb_meta_auc, test_lgb_meta),
    'Neural Network Meta-model': (nn_meta_auc, test_nn_meta),
    'Rank Average': (rank_avg_auc, test_rank_avg),
    'Performance Weighted': (weighted_auc, test_weighted),
    'Optimized Weights': (optimized_auc, test_optimized)
}

print("\nStacking Results:")
for name, (auc, _) in sorted(ensemble_results.items(), key=lambda x: x[1][0], reverse=True):
    improvement = auc - 0.6385
    symbol = "🎯" if auc >= 0.65 else "📈" if auc > 0.64 else "📊"
    print(f"{symbol} {name:30s}: {auc:.4f} ({improvement:+.4f})")

best_ensemble = max(ensemble_results, key=lambda x: ensemble_results[x][0])
best_auc, best_test_preds = ensemble_results[best_ensemble]

print(f"\n🏆 Best Ensemble: {best_ensemble}")
print(f"📊 OOF AUROC: {best_auc:.4f}")

if best_auc >= 0.65:
    print("\n🎉🎉🎉 TARGET REACHED! AUROC >= 0.65! 🎉🎉🎉")
else:
    gap = 0.65 - best_auc
    print(f"\n📈 Gap to target: {gap:.4f}")
    print(f"   Progress: 0.6385 → {best_auc:.4f} (+{best_auc - 0.6385:.4f})")


FINAL ENSEMBLE COMPARISON

Stacking Results:
📊 Ridge Logistic Regression     : 0.6370 (-0.0015)
📊 Optimized Weights             : 0.6370 (-0.0015)
📊 Neural Network Meta-model     : 0.6357 (-0.0028)
📊 Performance Weighted          : 0.6355 (-0.0030)
📊 LightGBM Meta-model           : 0.6351 (-0.0034)
📊 Rank Average                  : 0.6330 (-0.0055)

🏆 Best Ensemble: Ridge Logistic Regression
📊 OOF AUROC: 0.6370

📈 Gap to target: 0.0130
   Progress: 0.6385 → 0.6370 (+-0.0015)


In [9]:
# ========================================================================
# STEP 7: CREATE SUBMISSIONS
# ========================================================================
print("\n" + "=" * 80)
print("STEP 7: CREATING SUBMISSIONS")
print("=" * 80)

test_ids = test[ID_COL] if ID_COL in test.columns else range(len(test))

# Create submission for each ensemble method
submissions_created = []

for method_name, (oof_score, test_preds) in ensemble_results.items():
    submission = pd.DataFrame({
        'id': test_ids,
        'target': test_preds
    })
    
    filename = f'submission_neural_{method_name.lower().replace(" ", "_").replace("-", "_")}_{oof_score:.4f}.csv'
    submission.to_csv(filename, index=False)
    submissions_created.append((method_name, oof_score, filename))
    print(f"✅ {filename}")

# Also create submissions for top 3 base models
print("\n📦 Creating base model submissions...")
for name, preds in sorted(oof_predictions.items(), key=lambda x: roc_auc_score(y_full, x[1]), reverse=True)[:3]:
    auc = roc_auc_score(y_full, preds)
    test_preds = test_predictions[name]
    
    submission = pd.DataFrame({
        'id': test_ids,
        'target': test_preds
    })
    
    filename = f'submission_base_{name}_{auc:.4f}.csv'
    submission.to_csv(filename, index=False)
    print(f"✅ {filename}")


STEP 7: CREATING SUBMISSIONS
✅ submission_neural_ridge_logistic_regression_0.6370.csv
✅ submission_neural_lightgbm_meta_model_0.6351.csv
✅ submission_neural_neural_network_meta_model_0.6357.csv
✅ submission_neural_rank_average_0.6330.csv
✅ submission_neural_performance_weighted_0.6355.csv
✅ submission_neural_optimized_weights_0.6370.csv

📦 Creating base model submissions...
✅ submission_base_catboost_0.6360.csv
✅ submission_base_xgboost_0.6324.csv
✅ submission_base_lightgbm_0.6286.csv


In [10]:
# ========================================================================
# STEP 8: SAVE MODELS
# ========================================================================
print("\n" + "=" * 80)
print("STEP 8: SAVING MODELS")
print("=" * 80)

# Save best meta-model
if best_ensemble == 'Ridge Logistic Regression':
    joblib.dump(lr_cv, 'meta_model_ridge_lr.joblib')
    print("✅ Saved: meta_model_ridge_lr.joblib")
elif best_ensemble == 'LightGBM Meta-model':
    joblib.dump(lgb_meta, 'meta_model_lgb.joblib')
    print("✅ Saved: meta_model_lgb.joblib")
elif best_ensemble == 'Neural Network Meta-model':
    joblib.dump(meta_nn, 'meta_model_nn.joblib')
    print("✅ Saved: meta_model_nn.joblib")

# Save optimal weights
np.save('optimal_ensemble_weights.npy', opt_weights)
print("✅ Saved: optimal_ensemble_weights.npy")

# Save OOF and test predictions for future use
oof_df = pd.DataFrame(oof_predictions)
oof_df['target'] = y_full.values
oof_df.to_csv('oof_predictions_all_models.csv', index=False)
print("✅ Saved: oof_predictions_all_models.csv")

test_df = pd.DataFrame(test_predictions)
test_df['id'] = test_ids
test_df.to_csv('test_predictions_all_models.csv', index=False)
print("✅ Saved: test_predictions_all_models.csv")


STEP 8: SAVING MODELS
✅ Saved: meta_model_ridge_lr.joblib
✅ Saved: optimal_ensemble_weights.npy
✅ Saved: oof_predictions_all_models.csv
✅ Saved: test_predictions_all_models.csv


In [11]:
# ========================================================================
# STEP 9: DIVERSITY ANALYSIS
# ========================================================================
print("\n" + "=" * 80)
print("STEP 9: MODEL DIVERSITY ANALYSIS")
print("=" * 80)

# Calculate correlation between model predictions
print("\nPrediction Correlations (OOF):")
corr_matrix = pd.DataFrame(oof_predictions).corr()
print(corr_matrix.round(3))

print("\nDiversity Insights:")
avg_corr = corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean()
print(f"  Average correlation: {avg_corr:.3f}")

if avg_corr < 0.85:
    print("  ✅ Good diversity - models make different errors")
elif avg_corr < 0.90:
    print("  ⚠️  Moderate diversity - some redundancy")
else:
    print("  🚨 Low diversity - models too similar")

# Find least correlated pairs (good for ensembling)
print("\nLeast correlated model pairs (best for diversity):")
corr_pairs = []
for i in range(len(corr_matrix)):
    for j in range(i+1, len(corr_matrix)):
        corr_pairs.append((
            corr_matrix.index[i],
            corr_matrix.columns[j],
            corr_matrix.iloc[i, j]
        ))

for model1, model2, corr in sorted(corr_pairs, key=lambda x: x[2])[:5]:
    print(f"  {model1} <-> {model2}: {corr:.3f}")


STEP 9: MODEL DIVERSITY ANALYSIS

Prediction Correlations (OOF):
            catboost  lightgbm  xgboost  extratrees    mlp
catboost       1.000     0.901    0.929       0.848  0.646
lightgbm       0.901     1.000    0.933       0.759  0.601
xgboost        0.929     0.933    1.000       0.779  0.621
extratrees     0.848     0.759    0.779       1.000  0.669
mlp            0.646     0.601    0.621       0.669  1.000

Diversity Insights:
  Average correlation: 0.769
  ✅ Good diversity - models make different errors

Least correlated model pairs (best for diversity):
  lightgbm <-> mlp: 0.601
  xgboost <-> mlp: 0.621
  catboost <-> mlp: 0.646
  extratrees <-> mlp: 0.669
  lightgbm <-> extratrees: 0.759


In [12]:
# ========================================================================
# STEP 10: SUBMISSION STRATEGY & RECOMMENDATIONS
# ========================================================================
print("\n" + "=" * 80)
print("SUBMISSION STRATEGY")
print("=" * 80)

print("\n📋 Recommended Submission Order:")
print(f"\n1️⃣  PRIMARY: submission_neural_{best_ensemble.lower().replace(' ', '_').replace('-', '_')}_{best_auc:.4f}.csv")
print(f"   OOF AUROC: {best_auc:.4f}")
print(f"   Strategy: Best performing ensemble")

# Get second and third best
sorted_ensembles = sorted(ensemble_results.items(), key=lambda x: x[1][0], reverse=True)

if len(sorted_ensembles) > 1:
    second_name, (second_auc, _) = sorted_ensembles[1]
    print(f"\n2️⃣  BACKUP: submission_neural_{second_name.lower().replace(' ', '_').replace('-', '_')}_{second_auc:.4f}.csv")
    print(f"   OOF AUROC: {second_auc:.4f}")
    print(f"   Strategy: Fallback if primary underperforms")

if len(sorted_ensembles) > 2:
    third_name, (third_auc, _) = sorted_ensembles[2]
    print(f"\n3️⃣  ALTERNATIVE: submission_neural_{third_name.lower().replace(' ', '_').replace('-', '_')}_{third_auc:.4f}.csv")
    print(f"   OOF AUROC: {third_auc:.4f}")
    print(f"   Strategy: Different ensemble approach")

print("\n" + "=" * 80)
print("EXPECTED PERFORMANCE")
print("=" * 80)

print(f"\n📊 Performance Summary:")
print(f"   Starting point (previous): 0.6385")
print(f"   Current best (OOF):        {best_auc:.4f}")
print(f"   Improvement:               +{best_auc - 0.6385:.4f}")
print(f"   Target:                    0.6500")

if best_auc >= 0.65:
    print(f"\n✅ Target achieved locally!")
    print(f"   Expected Kaggle: {best_auc - 0.002:.4f} to {best_auc + 0.002:.4f}")
else:
    print(f"\n📈 Gap to target: {0.65 - best_auc:.4f}")
    print(f"   Expected Kaggle: {best_auc - 0.003:.4f} to {best_auc + 0.001:.4f}")

print("\n⚠️  Important Notes:")
print("   • OOF scores are more reliable than single validation splits")
print("   • Expected Kaggle score: OOF ± 0.003")
print("   • If Kaggle >> OOF: lucky public test set (watch for shake-up)")
print("   • If Kaggle << OOF: possible data leakage or overfitting")


SUBMISSION STRATEGY

📋 Recommended Submission Order:

1️⃣  PRIMARY: submission_neural_ridge_logistic_regression_0.6370.csv
   OOF AUROC: 0.6370
   Strategy: Best performing ensemble

2️⃣  BACKUP: submission_neural_optimized_weights_0.6370.csv
   OOF AUROC: 0.6370
   Strategy: Fallback if primary underperforms

3️⃣  ALTERNATIVE: submission_neural_neural_network_meta_model_0.6357.csv
   OOF AUROC: 0.6357
   Strategy: Different ensemble approach

EXPECTED PERFORMANCE

📊 Performance Summary:
   Starting point (previous): 0.6385
   Current best (OOF):        0.6370
   Improvement:               +-0.0015
   Target:                    0.6500

📈 Gap to target: 0.0130
   Expected Kaggle: 0.6340 to 0.6380

⚠️  Important Notes:
   • OOF scores are more reliable than single validation splits
   • Expected Kaggle score: OOF ± 0.003
   • If Kaggle >> OOF: lucky public test set (watch for shake-up)
   • If Kaggle << OOF: possible data leakage or overfitting


In [13]:
# ========================================================================
# STEP 11: ADVANCED TIPS FOR FURTHER IMPROVEMENT
# ========================================================================
print("\n" + "=" * 80)
print("FURTHER IMPROVEMENT STRATEGIES")
print("=" * 80)

if best_auc < 0.65:
    print("\n💡 To reach 0.65+, try:")
    print("   1. Add more diverse base models:")
    print("      • Different neural net architectures")
    print("      • Naive Bayes variants")
    print("      • SVM with RBF kernel")
    print("   ")
    print("   2. Advanced feature engineering:")
    print("      • Target encoding with different alphas")
    print("      • Frequency encoding")
    print("      • Clustering-based features")
    print("   ")
    print("   3. Two-level stacking:")
    print("      • Layer 1: Current base models")
    print("      • Layer 2: Ensemble layer 1 outputs")
    print("      • Layer 3: Final meta-model")
    print("   ")
    print("   4. Pseudo-labeling:")
    print("      • Use high-confidence test predictions")
    print("      • Retrain models with augmented data")
    print("   ")
    print("   5. Model calibration:")
    print("      • Calibrate probabilities with isotonic regression")
    print("      • Often improves ranking metrics")

print("\n" + "=" * 80)
print("KEY INSIGHTS FOR STACKING SUCCESS")
print("=" * 80)

print("\n✅ What makes good stacking:")
print("   • Diverse base models (low correlation)")
print("   • Different model types (trees, linear, NN)")
print("   • Conservative meta-model (prevent overfitting)")
print("   • Nested CV for meta-features (no leakage)")
print("   • Strong regularization")

print("\n❌ Common stacking mistakes:")
print("   • Too complex meta-model (overfits to OOF)")
print("   • High correlation between base models")
print("   • No regularization in meta-model")
print("   • Using validation predictions instead of OOF")
print("   • Too many base models (diminishing returns)")

print("\n📈 Your current setup:")
num_base_models = len(oof_predictions)
print(f"   • Base models: {num_base_models}")
print(f"   • Average correlation: {avg_corr:.3f}")
print(f"   • Best ensemble: {best_ensemble}")
print(f"   • Regularization: {'Strong' if 'Ridge' in best_ensemble or 'LightGBM' in best_ensemble else 'Moderate'}")

if num_base_models >= 5 and avg_corr < 0.85:
    print("\n   ✅ Good setup! Diverse models + proper regularization")
elif num_base_models < 5:
    print("\n   💡 Add more diverse models for better ensemble")
elif avg_corr >= 0.85:
    print("\n   💡 Models too correlated - add different model types")

# ========================================================================
# FINAL SUMMARY
# ========================================================================
print("\n" + "=" * 80)
print("🎯 PIPELINE COMPLETE - FINAL SUMMARY")
print("=" * 80)

print(f"\n📊 Results:")
print(f"   Previous best:  0.6385 (Linear Stacking)")
print(f"   Current best:   {best_auc:.4f} ({best_ensemble})")
print(f"   Improvement:    +{best_auc - 0.6385:.4f}")
print(f"   Target:         0.6500")
print(f"   Status:         {'✅ ACHIEVED' if best_auc >= 0.65 else f'📈 Gap: {0.65 - best_auc:.4f}'}")

print(f"\n📁 Files created:")
print(f"   • {len(submissions_created)} ensemble submissions")
print(f"   • 3 base model submissions")
print(f"   • OOF predictions CSV")
print(f"   • Test predictions CSV")
print(f"   • Optimal weights file")
print(f"   • Best meta-model")

print(f"\n🚀 Next steps:")
print(f"   1. Submit PRIMARY file to Kaggle")
print(f"   2. Compare Kaggle score to OOF score")
print(f"   3. If gap > 0.005: try more conservative models")
print(f"   4. If gap < 0.005: current approach is robust ✅")

print("\n" + "=" * 80)
print("✨ Good luck with your submission! ✨")
print("=" * 80)


FURTHER IMPROVEMENT STRATEGIES

💡 To reach 0.65+, try:
   1. Add more diverse base models:
      • Different neural net architectures
      • Naive Bayes variants
      • SVM with RBF kernel
   
   2. Advanced feature engineering:
      • Target encoding with different alphas
      • Frequency encoding
      • Clustering-based features
   
   3. Two-level stacking:
      • Layer 1: Current base models
      • Layer 2: Ensemble layer 1 outputs
      • Layer 3: Final meta-model
   
   4. Pseudo-labeling:
      • Use high-confidence test predictions
      • Retrain models with augmented data
   
   5. Model calibration:
      • Calibrate probabilities with isotonic regression
      • Often improves ranking metrics

KEY INSIGHTS FOR STACKING SUCCESS

✅ What makes good stacking:
   • Diverse base models (low correlation)
   • Different model types (trees, linear, NN)
   • Conservative meta-model (prevent overfitting)
   • Nested CV for meta-features (no leakage)
   • Strong regularization
